In [97]:
!pip install geopy
!pip install folium
import pandas as pd
import math
from geopy.distance import geodesic

In [98]:
# Load the charging stations data
charging_stations_df = pd.read_csv("Electric_Vehicle_Charging_Stations.csv")

# Load the places data
places_df = pd.read_csv("coordinates.csv")

# Concatenate "Station Name" and "Street Address" into a new column "Charging Stations"
charging_stations_df["Charging Stations"] = charging_stations_df["Station Name"] + ", " + charging_stations_df["Street Address"]
charging_stations_df = charging_stations_df.drop(columns=["Station Name", "Street Address"])

# Rename the "Places" column to "Locations" in places_df
places_df = places_df.rename(columns={"Places": "Locations"})

# Display the resulting DataFrames
print("Charging Stations DataFrame:")
print(charging_stations_df.head())
print("\nPlaces DataFrame:")
print(places_df.head())

Charging Stations DataFrame:
           City              Access Days Time EV Level1 EVSE Num  \
0        Darien                24 hours daily               NONE   
1       Meriden  24 hours, for Tesla use only               NONE   
2  Beacon Falls                24 hours daily               NONE   
3  Old Saybrook                24 hours daily               NONE   
4     Fairfield                24 hours daily               NONE   

  EV Level2 EVSE Num EV DC Fast Count  Longitude   Latitude  \
0                  2             NONE -73.476469  41.072882   
1               NONE                8 -72.773473  41.527367   
2                  1             NONE -73.065583  41.445481   
3                  2             NONE -72.382500  41.310278   
4                  2             NONE -73.264511  41.143125   

                                   Charging Stations  
0                    BMW OF DARIEN, 138-142 Ledge Rd  
1        Dunkin’ - Tesla Supercharger, 893 E Main St  
2  Town of Beacon 

In [99]:
# Create a distance matrix
distance_matrix = pd.DataFrame(index=charging_stations_df["Charging Stations"], columns=places_df["Locations"])

for idx, row in charging_stations_df.iterrows():
    for col in places_df["Locations"]:
        station_coords = (row["Latitude"], row["Longitude"])
        place_coords = (places_df.loc[places_df["Locations"] == col, "Latitude"].values[0],
                        places_df.loc[places_df["Locations"] == col, "Longitude"].values[0])
        distance = geodesic(station_coords, place_coords).miles
        distance_matrix.at[row["Charging Stations"], col] = distance

In [100]:

# Create a distance matrix with distance for qualifying charging stations
qualifying_charging_stations = charging_stations_df[
    (charging_stations_df["Access Days Time"].str.contains("24 hours")) &
    (
        (pd.to_numeric(charging_stations_df["EV Level2 EVSE Num"], errors='coerce') > 0) |
        (pd.to_numeric(charging_stations_df["EV DC Fast Count"], errors='coerce') > 0)
    )]

qualifying_distance_matrix = pd.DataFrame(index=qualifying_charging_stations["Charging Stations"], columns=places_df["Locations"])

for idx, row in qualifying_charging_stations.iterrows():
    for col in places_df["Locations"]:
        qualifying_distance_matrix.at[row["Charging Stations"], col] = distance_matrix.at[row["Charging Stations"], col]
print(qualifying_distance_matrix)

Locations                                          Connecticut State Capitol  \
Charging Stations                                                              
BMW OF DARIEN, 138-142 Ledge Rd                                    63.075025   
Dunkin’ - Tesla Supercharger, 893 E Main St                        17.016899   
Town of Beacon Falls - Commuter Lot, 105 N Main St                 29.634424   
OLD SAYBROOK VW, 319 Middlesex Turnpike                            34.974589   
Fairfield Rail Station, 80 Mill Plain Rd                           52.451869   
...                                                                      ...   
Yankee Doodle, 10 Burnell Boulevard                                58.620315   
777 MAIN ST, 71 Asylum St                                            0.43666   
Norwich Public Utilities, 173 N Main St                            35.955034   
Fairfield Plaza, 116 Danbury Rd                                    40.647661   
Eastern Connecticut State University - S

In [101]:
# Create a location distance matrix
location_distance_matrix = pd.DataFrame(index=places_df["Locations"], columns=places_df["Locations"])

for idx1, row1 in places_df.iterrows():
    for idx2, row2 in places_df.iterrows():
        location1_coords = (row1["Latitude"], row1["Longitude"])
        location2_coords = (row2["Latitude"], row2["Longitude"])
        distance = geodesic(location1_coords, location2_coords).miles
        location_distance_matrix.at[row1["Locations"], row2["Locations"]] = distance

# Display the location distance matrix
print("Location Distance Matrix:")
print(location_distance_matrix)

Location Distance Matrix:
Locations                                   Connecticut State Capitol  \
Locations                                                               
Connecticut State Capitol                                         0.0   
Pratt & Whitney Stadium at Rentschler Field                  3.305513   
Talcott Mountain State Park                                  7.459013   
New England Air Museum                                      12.670356   
Phelps-Hatheway House & Garden                              14.984078   
Connecticut Trolley Museum                                  12.426005   
Shenipsit State Forest                                      19.631927   
University of Connecticut                                   22.325602   
Patriots Park                                               19.411611   
Wadsworth Falls State Park                                  15.691763   
Hubbard Park                                                16.901365   
Wharton Brook State Park 

In [102]:
def calculate_battery_discharge_rate(distance, speed, Cd, A, Crr, m, g, rho, lambda_, alpha):
    # Calculate the battery discharge rate based on vehicle parameters
    F_drag = 0.5 * rho * Cd * A * (speed ** 2)
    F_rolling = Crr * m * g
    F_total = F_drag + F_rolling
    P_total = F_total * speed
    P_regen = alpha * P_total
    P_net = P_total - P_regen
    energy_consumed = P_net * (distance / speed)
    discharge_rate = energy_consumed / 3600  # Convert from Joules to kWh
    return discharge_rate

In [103]:
def find_nearest_city(current_tour, location_distance_matrix, places_df, qualifying_distance_matrix):
    print("Finding nearest city...")
    nearest_city = None
    min_distance = float('inf')

    for location in places_df["Locations"]:
        if location not in current_tour:
            if location in qualifying_distance_matrix.columns:
                for station in qualifying_distance_matrix.index:
                    distance = qualifying_distance_matrix.at[station, location]
                    if distance < min_distance:
                        nearest_city = location
                        min_distance = distance
            else:
                for city in current_tour:
                    if city in location_distance_matrix.index:
                        if location_distance_matrix.at[city, location] < min_distance:
                            nearest_city = location
                            min_distance = location_distance_matrix.at[city, location]
                    else:
                        print(f"City '{city}' not found in the location distance matrix. Skipping distance calculation.")

    print("Nearest city found:", nearest_city)
    return nearest_city

In [104]:
def find_nearest_charging_station(current_station, qualifying_distance_matrix, location_distance_matrix, places_df):
    print("Finding nearest charging station...")
    nearest_station = None
    min_distance = float('inf')

    if current_station in qualifying_distance_matrix.columns:
        for station in qualifying_distance_matrix.index:
            distance = qualifying_distance_matrix.at[station, current_station]
            if distance < min_distance:
                nearest_station = station
                min_distance = distance
    else:
        if current_station in location_distance_matrix.index:
            for location in places_df["Locations"]:
                if location in qualifying_distance_matrix.columns:
                    distance = location_distance_matrix.at[current_station, location]
                    if distance < min_distance:
                        nearest_station = location
                        min_distance = distance
        else:
            print(f"Location '{current_station}' not found in either distance matrix.")

    if nearest_station is not None:
        print("Nearest charging station found:", nearest_station)
    else:
        print("No suitable charging station found.")

    return nearest_station

In [105]:
def insert_nearest_city(nearest_city, current_tour, location_distance_matrix, qualifying_distance_matrix):
    print("Inserting nearest city...")
    best_insertion = None
    min_increase = float('inf')
    
    for i in range(len(current_tour)):
        if current_tour[i] in location_distance_matrix.index:
            prev_city = current_tour[i]
        else:
            prev_city = current_tour[i-1]
        
        next_i = (i + 1) % len(current_tour)
        
        if current_tour[next_i] in location_distance_matrix.columns:
            next_city = current_tour[next_i]
            increase = location_distance_matrix.at[prev_city, nearest_city] + location_distance_matrix.at[nearest_city, next_city] - location_distance_matrix.at[prev_city, next_city]
        else:
            next_city = current_tour[0]
            if current_tour[i] in qualifying_distance_matrix.index and nearest_city in qualifying_distance_matrix.columns:
                increase = qualifying_distance_matrix.at[current_tour[i], nearest_city]
            else:
                increase = float('inf')
            
            if next_city in location_distance_matrix.columns:
                increase += location_distance_matrix.at[nearest_city, next_city]
            else:
                increase = float('inf')
            
            if current_tour[i] in qualifying_distance_matrix.index and next_city in qualifying_distance_matrix.columns:
                increase -= qualifying_distance_matrix.at[current_tour[i], next_city]
            else:
                increase = float('inf')
        
        if increase < min_increase:
            min_increase = increase
            best_insertion = i + 1
    
    current_tour.insert(best_insertion, nearest_city)
    print("Nearest city inserted into the tour.")




In [106]:
def nearest_insertion_algorithm(location_distance_matrix, qualifying_distance_matrix, places_df, charging_stations_df, source_city, vehicle_params, charging_params, iteration_count=0):
    current_tour = [source_city]
    visited_cities = set(current_tour)
    current_charge = charging_params['max_charge']
    total_distance = 0

    Cd, A, Crr, m, g, rho, lambda_, alpha = vehicle_params.values()
    level2_charge_rate, dc_fast_charge_rate, min_charge, max_charge = charging_params.values()

    while len(visited_cities) < len(places_df):
        if current_tour[-1] in places_df["Locations"].values:
            nearest_city = find_nearest_city(current_tour, location_distance_matrix, places_df, qualifying_distance_matrix)
            distance_to_next_city = location_distance_matrix.at[current_tour[-1], nearest_city]
        else:
            nearest_city = find_nearest_city(current_tour[:-1], location_distance_matrix, places_df, qualifying_distance_matrix)
            distance_to_next_city = qualifying_distance_matrix.at[current_tour[-1], nearest_city]

        speed = 50  # Assume an average speed of  (50 km/h)
        discharge_rate = calculate_battery_discharge_rate(distance_to_next_city, speed, Cd, A, Crr, m, g, rho, lambda_, alpha)
        required_charge_to_next_city = discharge_rate

        if current_charge - required_charge_to_next_city >= min_charge:
            insert_nearest_city(nearest_city, current_tour, location_distance_matrix, qualifying_distance_matrix)
            visited_cities.add(nearest_city)
            current_charge -= required_charge_to_next_city
            total_distance += distance_to_next_city
        else:
            nearest_station = find_nearest_charging_station(current_tour[-1], qualifying_distance_matrix, location_distance_matrix, places_df)

            if nearest_station is not None:
                distance_to_station = qualifying_distance_matrix.at[nearest_station, current_tour[-1]]
                discharge_rate_to_station = calculate_battery_discharge_rate(distance_to_station, speed, Cd, A, Crr, m, g, rho, lambda_, alpha)
                required_charge_to_station = discharge_rate_to_station

                if current_charge - required_charge_to_station >= min_charge:
                    current_tour.append(nearest_station)
                    if 'DC Fast' in nearest_station:
                        charge_rate = dc_fast_charge_rate
                    else:
                        charge_rate = level2_charge_rate
                    charging_time = (max_charge - current_charge) / charge_rate
                    current_charge = max_charge
                    total_distance += distance_to_station
                else:
                    print(f"Impossible route encountered after {iteration_count} iterations. Restarting with a different source city.")
                    new_source_city = places_df.loc[places_df["Locations"] != source_city, "Locations"].sample().values[0]
                    return nearest_insertion_algorithm(location_distance_matrix, qualifying_distance_matrix, places_df, charging_stations_df, new_source_city, vehicle_params, charging_params, iteration_count + 1)
            else:
                print(f"Impossible route encountered after {iteration_count} iterations. Restarting with a different source city.")
                new_source_city = places_df.loc[places_df["Locations"] != source_city, "Locations"].sample().values[0]
                return nearest_insertion_algorithm(location_distance_matrix, qualifying_distance_matrix, places_df, charging_stations_df, new_source_city, vehicle_params, charging_params, iteration_count + 1)

    print(f"Nearest Insertion Algorithm completed after {iteration_count} iterations.")
    print("Tour:")
    for city in current_tour:
        if city in places_df["Locations"].values:
            print(f"Nearest City: {city}")
        else:
            print(f"Nearest Charging Station: {city}")


    if all(location in current_tour for location in places_df["Locations"]):
        print("Feasible route found.")
        print("Best Cost (Total Distance):", total_distance)
        return current_tour, total_distance
    else:
        print(f"Impossible route encountered after {iteration_count} iterations. Restarting with a different source city.")
        new_source_city = places_df.loc[places_df["Locations"] != source_city, "Locations"].sample().values[0]
        return nearest_insertion_algorithm(location_distance_matrix, qualifying_distance_matrix, places_df, charging_stations_df, new_source_city, vehicle_params, charging_params, iteration_count + 1)
vehicle_params = {
    'Cd': 0.3,
    'A': 2.2,
    'Crr': 0.01,
    'm': 1500,
    'g': 9.81,
    'rho': 1.225,
    'lambda_': 0.80,
    'alpha': 0.25
}

charging_params = {
    'level2_charge_rate': 7,
    'dc_fast_charge_rate': 50,
    'min_charge': 20,  # 20% of 100 kWh battery capacity
    'max_charge': 90  # 90% of 100 kWh battery capacity
}

These values represent various parameters and coefficients used in vehicle dynamics and aerodynamics calculations. Let's go through each one:

1. 'Cd': 0.3 stands for the coefficient of drag, which is a dimensionless quantity that represents the aerodynamic drag of an object.
2. 'A': 2.2 represents the frontal area of the vehicle in square meters (m^2).
3. 'Crr': 0.01 stands for the coefficient of rolling resistance, which is a dimensionless quantity that represents the resistance to motion of the vehicle due to friction between the tires and the road surface.
4. 'm': 1500 represents the mass of the vehicle in kilograms (kg).
5. 'g': 9.81 represents the acceleration due to gravity in meters per second squared (m/s^2).
6. 'rho': 1.225 represents the density of air in kilograms per cubic meter (kg/m^3).
7. 'lambda_': 0.80 is a coefficient related to the efficiency of the vehicle's powertrain.
8. 'alpha': 0.25 is a coefficient related to the road gradient or slope.


In [107]:
source_city = "Yale University"

while True:
    print("Starting tour with source city:", source_city)
    tour, total_distance = nearest_insertion_algorithm(location_distance_matrix, distance_matrix, places_df, charging_stations_df, source_city, vehicle_params, charging_params)

    if tour is not None:
        print("Tour:")
        for city in tour:
            if city in places_df["Locations"].values:
                print(f"Nearest City: {city}")
            else:
                print(f"Nearest Charging Station: {city}")
        print("Best Cost (Total Distance):", total_distance)
        break
    else:
        source_city = places_df.loc[places_df["Locations"] != source_city, "Locations"].sample().values[0]

Starting tour with source city: Yale University
Finding nearest city...
Nearest city found: Connecticut State Capitol
Inserting nearest city...
Nearest city inserted into the tour.
Finding nearest city...
Nearest city found: The Maritime Aquarium
Inserting nearest city...
Nearest city inserted into the tour.
Finding nearest city...
Nearest city found: Bruce Museum
Inserting nearest city...
Nearest city inserted into the tour.
Finding nearest city...
Nearest city found: Rotary Park & Bandstand
Inserting nearest city...
Nearest city inserted into the tour.
Finding nearest city...
Nearest city found: University of Connecticut
Inserting nearest city...
Nearest city inserted into the tour.
Finding nearest city...
Nearest city found: Westport Country Playhouse
Finding nearest charging station...
Nearest charging station found: Cargill Chevrolet, 23 Livery St
Finding nearest city...
Nearest city found: Westport Country Playhouse
Inserting nearest city...
Nearest city inserted into the tour.
F

In [110]:
import folium


# Create a map centered on Connecticut
m = folium.Map(location=[41.6032, -73.0877], zoom_start=8)

# Create a feature group for markers
marker_group = folium.FeatureGroup(name='Markers')

# Add markers for each location in the tour
for location in tour:
    if location in places_df['Locations'].values:
        lat = places_df.loc[places_df['Locations'] == location, 'Latitude'].values[0]
        lon = places_df.loc[places_df['Locations'] == location, 'Longitude'].values[0]
        marker_group.add_child(folium.Marker([lat, lon], popup=location, icon=folium.Icon(color='blue')))
    elif location in charging_stations_df['Charging Stations'].values:
        lat = charging_stations_df.loc[charging_stations_df['Charging Stations'] == location, 'Latitude'].values[0]
        lon = charging_stations_df.loc[charging_stations_df['Charging Stations'] == location, 'Longitude'].values[0]
        marker_group.add_child(folium.Marker([lat, lon], popup=location, icon=folium.Icon(color='red')))
    else:
        print(f"Coordinates not found for location: {location}")

# Add a polyline for the tour path
# Combine the places and charging stations dataframes for easy lookup
combined_df = pd.concat([
    places_df[['Locations', 'Latitude', 'Longitude']].rename(columns={'Locations': 'Name'}),
    charging_stations_df[['Charging Stations', 'Latitude', 'Longitude']].rename(columns={'Charging Stations': 'Name'})
])

# Ensure the 'Name' column is of string type to avoid any potential matching issues
combined_df['Name'] = combined_df['Name'].astype(str)

# Use list comprehension to build the path based on the 'tour' list
path = [
    combined_df.loc[combined_df['Name'] == location, ['Latitude', 'Longitude']].values.flatten().tolist()
    for location in tour
    if combined_df['Name'].str.contains(location).any()
]

polyline_group = folium.FeatureGroup(name='Polylines')
polyline_group.add_child(folium.PolyLine(path, color='red', weight=2))

# Add feature groups to the map
m.add_child(marker_group)
m.add_child(polyline_group)

# Add layer control to the map
folium.LayerControl().add_to(m)

# Display the map
m